# Modelo Quantizado para Reconhecimento de Anel de Sinete

Este notebook gera um modelo quantizado para classificação de tecido com ou sem anel de sinete. Tal modelo é utilizado na geração do mapa de calor para a WSI.


In [ ]:
# A biblioteca a seguir não é padrão do google colab, por isso é necessário
# instala-la antes de executar a célula de importação das bibliotecas
!pip install tensorflow_model_optimization

In [6]:
import tensorflow_model_optimization as tfmot

import os
import pathlib
import warnings
import platform

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import layers, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.metrics import classification_report, confusion_matrix

In [7]:
print("Python- Versão: ", platform.python_version())
print("TensorFLow - Versão: ",tf.__version__)

# Checar GPUs
if not tf.test.gpu_device_name():
    warnings.warn('GPU não encontrada.')
else:
    print(f'Dispositivo GPU padrão: {tf.test.gpu_device_name()}')
    print("Número de GPUs Disponíveis: ", len(tf.config.list_physical_devices('GPU')))

Python- Versão:  3.10.12
TensorFLow - Versão:  2.12.0
Dispositivo GPU padrão: /device:GPU:0
Número de GPUs Disponíveis:  1


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# extrai o dataset de anel de sinete
!tar -xzvf "/content/drive/MyDrive/1_ DESENVOLVIMENTO  AMPLIAR TEC 4.0/Datasets/Anel de Sinete/TCGA-BR-8291-dataset-sinet-ring.tar.gz"

In [10]:
for dirpath, dirnames, filenames in os.walk('dataset-sinet-ring'):
    if len(filenames)!=0:
        print(f'Existem {len(filenames)} imagens em  {dirpath}.')

Existem 2760 imagens em  dataset-sinet-ring/train/com anel.
Existem 3630 imagens em  dataset-sinet-ring/train/sem anel.
Existem 86 imagens em  dataset-sinet-ring/val/com anel.
Existem 90 imagens em  dataset-sinet-ring/val/sem anel.
Existem 86 imagens em  dataset-sinet-ring/test/com anel.
Existem 90 imagens em  dataset-sinet-ring/test/sem anel.


In [11]:
train_dir = 'dataset-sinet-ring/train'
val_dir = 'dataset-sinet-ring/val'
test_dir = 'dataset-sinet-ring/test'

# Especificações para o treinamento da rede
IMG_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
EPOCHS = 100

In [12]:
train_datagen = ImageDataGenerator(rescale=None)
valid_datagen = ImageDataGenerator(rescale=None)
test_datagen = ImageDataGenerator(rescale=None)

train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=BATCH_SIZE,
                                               target_size=(224, 224),
                                               class_mode="categorical")

val_data = valid_datagen.flow_from_directory(val_dir,
                                               batch_size=BATCH_SIZE,
                                               target_size=(224, 224),
                                               class_mode="categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=BATCH_SIZE,
                                               target_size=(224, 224),
                                               class_mode="categorical")

Found 6390 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


In [15]:
# Criação do modelo

base_model = tf.keras.applications.MobileNetV2(include_top=True,
                                               input_shape=IMG_SHAPE,
                                               weights='imagenet',
                                               pooling='max')

# descongela as duas últimas camadas do modelo
base_output = base_model.layers[-2]

outputs = tf.keras.layers.Dense(2, activation='softmax')(base_output.output)

qat_model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

qat_model = tfmot.quantization.keras.quantize_model(qat_model)

## Fine Tuning - Fase 1 (warm up) -> treina apenas a camada densa

In [17]:
# Congela as camadas do modelo base (as camadas de convolução não serão treinadas novamente)
for layer in base_model.layers:
    layer.trainable = False

In [19]:
base_learning_rate = 0.0001
opt = Adam(learning_rate=base_learning_rate)
# binary -> binary_crossentropy; categorical -> categorical_crossentropy
metrics = ['acc', tf.keras.metrics.AUC()]
qat_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=metrics)

In [20]:
# Early Stopping do trinamento
early_stop = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta=0.001,
    patience=7
)

model_ckpt = keras.callbacks.ModelCheckpoint(
    filepath='callbacks/melhor_modelo_MobileNetV2_ft.hdf5',
    monitor='val_loss',
    save_best_only=True
)

callbacks = [early_stop, model_ckpt]

In [21]:
# Fase 1 - treina apenas com algumas épocas (40% do total)
initial_epochs = int(EPOCHS*0.4)
history = qat_model.fit(train_data,
                    epochs=initial_epochs,
                    steps_per_epoch=len(train_data),
                    validation_data=val_data,
                    validation_steps=len(val_data),
                    callbacks=callbacks)

Epoch 1/40
200/200 [==============================] - 137s 425ms/step - loss: 0.2939 - acc: 0.8684 - auc: 0.9474 - val_loss: 1.1307 - val_acc: 0.5568 - val_auc: 0.5547
Epoch 2/40
200/200 [==============================] - 75s 376ms/step - loss: 0.1159 - acc: 0.9559 - auc: 0.9919 - val_loss: 2.0988 - val_acc: 0.5568 - val_auc: 0.5959
Epoch 3/40
200/200 [==============================] - 79s 397ms/step - loss: 0.0610 - acc: 0.9770 - auc: 0.9978 - val_loss: 4.6464 - val_acc: 0.5000 - val_auc: 0.5142
Epoch 4/40
200/200 [==============================] - 77s 383ms/step - loss: 0.0278 - acc: 0.9914 - auc: 0.9996 - val_loss: 3.5558 - val_acc: 0.5170 - val_auc: 0.5719
Epoch 5/40
200/200 [==============================] - 75s 377ms/step - loss: 0.0268 - acc: 0.9906 - auc: 0.9995 - val_loss: 1.6003 - val_acc: 0.6648 - val_auc: 0.7314
Epoch 6/40
200/200 [==============================] - 75s 376ms/step - loss: 0.0149 - acc: 0.9948 - auc: 0.9999 - val_loss: 5.8998 - val_acc: 0.5341 - val_auc: 0.54

## Fine Tuning - Fase 2 -> treina todo o modelo do fine tuning - descongela as camadas de convolução

In [ ]:
print("Número de camadas do modelo base: ", len(basemodel.layers))

Número de camadas do modelo base:  158


In [22]:
# Descongelar as 30% camadas finais e que não seja do tipo BatchNormalization do modelo base
# Note que vpcê pode adicionar mais camadas se o modelo não for muito grande
count = 0
for layer in base_model.layers[-int(len(base_model.layers)*0.3):]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

In [23]:
# É necessário diminuir a base_learning_rate para que o modelo não tenha convergência muito rápida
# pois se trata d eum modelo grande
opt = Adam(learning_rate=base_learning_rate/10)
# binary -> binary_crossentropy; categorical -> categorical_crossentropy
qat_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=metrics)

In [24]:
# EarlyStopping do trinamento
early_stop2 = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta=0.001,
    patience=7
)

model_ckpt2 = keras.callbacks.ModelCheckpoint(
    filepath='callbacks/melhor_modelo_MobileNetV2_ft.hdf5',
    monitor='val_loss',
    save_best_only=True
)

callbacks2 = [early_stop2, model_ckpt2]

In [25]:
# Fase 2 - treina apenas com a quantidade de épocas que faltam (60% finais)
# Note que iniciará de initial_epoch=history.epoch[-1]
history_fine = qat_model.fit(train_data,
                    epochs=EPOCHS,
                    initial_epoch=history.epoch[-1],
                    steps_per_epoch=len(train_data),
                    validation_data=val_data,
                    validation_steps=len(val_data),
                    callbacks=callbacks2)

Epoch 22/100
200/200 [==============================] - 112s 386ms/step - loss: 0.0026 - acc: 0.9995 - auc: 0.9989 - val_loss: 0.5937 - val_acc: 0.9148 - val_auc: 0.9413
Epoch 23/100
200/200 [==============================] - 77s 383ms/step - loss: 0.0043 - acc: 0.9989 - auc: 0.9998 - val_loss: 0.4775 - val_acc: 0.9261 - val_auc: 0.9522
Epoch 24/100
200/200 [==============================] - 75s 375ms/step - loss: 0.0020 - acc: 0.9994 - auc: 1.0000 - val_loss: 0.4140 - val_acc: 0.9318 - val_auc: 0.9572
Epoch 25/100
200/200 [==============================] - 76s 380ms/step - loss: 0.0023 - acc: 0.9994 - auc: 1.0000 - val_loss: 0.3548 - val_acc: 0.9375 - val_auc: 0.9589
Epoch 26/100
200/200 [==============================] - 77s 382ms/step - loss: 0.0020 - acc: 0.9994 - auc: 1.0000 - val_loss: 0.3543 - val_acc: 0.9375 - val_auc: 0.9641
Epoch 27/100
200/200 [==============================] - 76s 379ms/step - loss: 9.1971e-04 - acc: 0.9998 - auc: 1.0000 - val_loss: 0.3757 - val_acc: 0.9261

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [27]:
# salvando modelo quantizado
tflite_model_name = 'TCGA-BR-8291_qat_sinete_model_1.tflite'
open(tflite_model_name, "wb").write(quantized_tflite_model)

2710576

In [ ]:
!mv TCGA-CD-A489_qat_sinete_model_1.tflite '/content/drive/MyDrive/1_ DESENVOLVIMENTO  AMPLIAR TEC 4.0/Notebooks DL/'

mv: cannot overwrite directory '/content/drive/MyDrive/1_ DESENVOLVIMENTO  AMPLIAR TEC 4.0/Notebooks DL/TCGA-CD-A489_qat_sinete_model_1.tflite' with non-directory
